# ***Análisis de Datos y Big data***

## ***Universidad de los Andes***
>## **Dirección de Educación Continua.**
>>## **Formación de habilidades escenciales.**

![Imágen1](https://github.com/jazaineam1/Andes_ADDBD/blob/main/Imagenes/Presentaci%C3%B3n-removebg-preview.png?raw=true)

**Ver la presentación  [acá](https://github.com/jazaineam1/Andes_ADDBD/raw/main/Sesion15/15_Agrupacion.pdf)**
# Clase 15
#  Aprendizaje no supervisado
## Agrupación K-medias, jerárquica y DBSCAN

Seguimos con la base de datos de consumidores de ayer.

In [2]:
!pip install plotly

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB 640.0 kB/s eta 0:00:26
   ---------------------------------------- 0.2/16.4 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.5/16.4 MB 3.8 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/16.4 MB 4.9 MB/s eta 0:00:04
   -- ------------------------------------- 1.2/16.4 MB 5.3 MB/s eta 0:00:03
   --- ------------------------------------ 1.3/16.4 MB 5.1 MB/s eta 0:00:03
   --- ------------------------------------ 1.6/16.4 MB 5.1 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/16.4 MB 5.2 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/16.4 MB 5.2 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/16.4 MB 5.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.6/16.4 MB 5.4 MB/s eta 0:00:03
   ------ --------------------------------- 2.7/16.4 MB 5.1 MB/s eta 0:00:03
   -

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly as py
import plotly.graph_objs as go

from sklearn.cluster import KMeans

from sklearn.cluster import DBSCAN

from scipy.cluster.hierarchy import linkage, fcluster
from scipy.cluster.hierarchy import dendrogram

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'plotly'

## Datos
Tenemos una base de datos: la unidad de observación son consumidores de un centro comercial.

In [ ]:
df = pd.read_csv('Mall_Customers.csv')
df.head()

: 

### Visualicemos esos consumidores

In [ ]:
plt.figure(1 , figsize = (15 , 7))
plt.title('Scatter plot of Age v/s Spending Score', fontsize = 20)
plt.xlabel('Age')
plt.ylabel('Spending Score')
plt.scatter( x = 'Age', y = 'Spending Score (1-100)', data = df, s = 100)
plt.show()

: 

## Modelos
Vamos a crear los siguientes modelos de agrupación y los vamos a comparar:
* K-medias
* Agrupación jerárquica
* Agrupación por densidad DBSCAN

### K-means
En K-medias, los centros se mueven hacia los datos cercanos, y luego los datos cercanos escogen su centro más cercano, hasta que hay estabilidad.

In [ ]:
X1 = df[['Age' , 'Spending Score (1-100)']].iloc[: , :].values
X1[0:5]

: 

### Análisis del codo

In [ ]:
X1 = df[['Age' , 'Spending Score (1-100)']].iloc[: , :].values
inertia = []
for n in range(1 , 15):
    algorithm = (KMeans(n_clusters = n ,init='k-means++', n_init = 10 ,max_iter=300,
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
    algorithm.fit(X1)
    inertia.append(algorithm.inertia_)

: 

In [ ]:
plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 15) , inertia , 'o')
plt.plot(np.arange(1 , 15) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()

: 

**Observamos que** parece que la incercia se estabiliza al rededor de cuatro grupos. Este puede ser un punto de partida, **pero no camisa de fuerza**, el criterio del analista es clave.

In [ ]:
## Después de haber hecho el análisis del codo, nos quedábamos con 4 grupos:

algorithm = (KMeans(n_clusters = 4 ,init='k-means++', n_init = 10 ,max_iter=300,
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
algorithm.fit(X1)
labels1 = algorithm.labels_
centroids1 = algorithm.cluster_centers_

: 

In [ ]:
plt.figure(1 , figsize = (15 , 7) )
plt.clf()
plt.scatter( x = 'Age', y = 'Spending Score (1-100)', data = df, c = labels1, s = 100)
#plt.scatter(x = centroids1[: , 0] , y =  centroids1[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Spending Score (1-100)') , plt.xlabel('Age')
plt.title('Agrupación por K-means')
plt.show()

: 

### Jerárquica
[Referencia](https://colab.research.google.com/github/goodboychan/chans_jupyter/blob/master/_notebooks/2020-06-06-02-Hierarchical-Clustering.ipynb#scrollTo=CdHHHWloEbov)

In [ ]:
# Formato correcto de los datos
X = df[['Age' , 'Spending Score (1-100)']]
X.head()

: 

In [ ]:
# Use the linkage()
distance_matrix = linkage(X, method='single', metric='euclidean')

# Assign cluster labels
numero_grupos = 5
# maxclust: En cuentra la distancia óptima entre dos observaciones para la creación de grupos
X['Grupo'] = fcluster(distance_matrix, numero_grupos, criterion='maxclust')

# Plot clusters
plt.figure(2 , figsize = (15 , 7) )
plt.title('Agrupación Jerárquica')
sns.scatterplot(x='Age', y='Spending Score (1-100)', s=100, hue='Grupo', data=X, palette='colorblind')

: 

In [ ]:
plt.title('Dendograma para 5 grupos')
dendrogram(distance_matrix)

: 

### DBSCAN

In [ ]:
# Usamos este formato
X1[0:5]

: 

In [ ]:
# eps: distancia máxima
# min_samples: número de datos para ser popular
modeloDensidad = DBSCAN(eps=6, min_samples=3).fit(X)
etiquetas = modeloDensidad.labels_
modeloDensidad

: 

In [ ]:
plt.figure(3 , figsize = (15 , 7) )
plt.title('Agrupación por DBSCAN')
c= sns.scatterplot(x='Age', y='Spending Score (1-100)', s=100, hue=etiquetas, data=X, palette='colorblind')
c.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

: 